In [1]:
import re

# LIBS IMPORT
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# READ DATASET
df = pd.read_csv("../../dataset/data_vivareal_21_03_2022_18_38_14.csv").drop(0)

In [3]:
df.head()

,id,url,address,area,rooms,batrooms,garage,amenities,value,condominium,date
1,2553339688,https://www.vivareal.com.br/imovel/apartamento...,"Rua Monsenhor Manoel Vicente, 597 - Água Verde...",51 m²,2 Quartos,1 Banheiro,1 Vaga,Aceita animais\nCozinha americana\nMobiliado\n...,R$ 325.000,Condomínio: R$ 350,2022-03-21 18:38:38
2,2074895474,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Professor Guido Straube, 775 - Vila Izabel...",72-78 m²,3 Quartos,2 Banheiros,2 Vagas,NaN,R$ 598.000,NaN,2022-03-21 18:38:38
3,2540019474,https://www.vivareal.com.br/imovel/apartamento...,"Rua Monsenhor Manoel Vicente, 316 - Água Verde...",94 m²,3 Quartos,2 Banheiros,2 Vagas,Mobiliado\nChurrasqueira\nElevador\nCondomínio...,R$ 680.000,Condomínio: R$ 530,2022-03-21 18:38:38
4,2518442439,https://www.vivareal.com.br/imovel/apartamento...,"Rua Coronel Dulcídio, 1756 - Água Verde, Curit...",54 m²,2 Quartos,1 Banheiro,-- Vaga,Mobiliado\nPortão eletrônico\nAceita animais\n...,R$ 215.000,Condomínio: R$ 345,2022-03-21 18:38:39
5,2477728884,https://www.vivareal.com.br/imovel/apartamento...,"Avenida Iguaçu, 2960 - Água Verde, Curitiba - PR",102 m²,3 Quartos,3 Banheiros,2 Vagas,Piscina\nMobiliado\nChurrasqueira\nElevador\nC...,R$ 650.000,Condomínio: R$ 1.000,2022-03-21 18:38:39


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36617 entries, 1 to 36617
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           36617 non-null  object
 1   url          36617 non-null  object
 2   address      36617 non-null  object
 3   area         36617 non-null  object
 4   rooms        36617 non-null  object
 5   batrooms     36617 non-null  object
 6   garage       36617 non-null  object
 7   amenities    26952 non-null  object
 8   value        36617 non-null  object
 9   condominium  16795 non-null  object
 10  date         36617 non-null  object
dtypes: object(11)
memory usage: 3.1+ MB


In [11]:
df.duplicated(subset=["id", "address","area", "rooms", "value"]).sum()

5949

In [12]:
df = df.drop_duplicates(subset=["id", "address","area", "rooms", "value"], keep="first")
df = df.reset_index(drop=True)

# Cleaning data

In [13]:
def cleanData(df, feature, split_character, idx):
    return df[feature].apply(lambda x : x.split(split_character)[idx])

In [18]:
df = df[~(df["area"] == "area")]
df = df.reset_index(drop=True)


## Area
51m^2 -> 51

In [59]:
# The area column have the m^2 string, remove it and let only the number
# 51m^2 -> 51
df["area"] = cleanData(df, "area", " ", 0)

# EXISTS SOME NEW APARTMENTS WITH area WITH TWO NUMBERS: 72-78, I WILL GET THE SMALL VALUE BECAUSE THE APARTMENT VALUE INIT 
# THE SMALL VALUE
df["area"] = cleanData(df, "area", "-", 0)


## Rooms

In [60]:
# rooms:
# 2 Quartos -> 2
# And remove strange rooms lines
df["rooms"] = cleanData(df, "rooms", " ", 0)
df[["rooms"]].value_counts()

rooms
3        14011
2         9994
1         3973
4         2488
5           96
2-3         41
3-4         18
1-3         15
1-2         13
6           10
--           2
2-4          2
20           2
12           1
19           1
dtype: int64

In [61]:
df["rooms"] = cleanData(df, "rooms", "-", 0)
df = df.loc[~(df["rooms"] == "rooms") & ~(df["rooms"] == "")]
df[["rooms"]].value_counts()
df["rooms"] = df["rooms"].astype(int)
# WE CAN SEE SOME OUTLIERS, A HOUSE WITH 20 ROOMS IS STRANGE. AFTER WE WILL REMOVE ALL THE OUTLIERS

## Batrooms

In [62]:
df["batrooms"] = cleanData(df, "batrooms", " ", 0)
df["batrooms"] = cleanData(df, "batrooms", "-", 0)
df = df.loc[~(df["batrooms"] == "")]

In [63]:
df["batrooms"].value_counts()
# HERE WE HAVE OUTLIER TO

2     10274
1      9934
3      4705
4      2592
5      2141
6       713
7       138
8        90
9        27
10       27
11       16
16        3
20        2
Name: batrooms, dtype: int64

## Garage

In [64]:
df["garage"] = cleanData(df, "garage", " ", 0)
df["garage"] = cleanData(df, "garage", "-", 0)

In [65]:
df["garage"].value_counts()

1     13426
2      8390
       4645
3      2578
4      1339
5       215
6        45
10        5
8         3
7         3
52        2
9         2
12        1
40        1
11        1
14        1
80        1
30        1
18        1
16        1
36        1
Name: garage, dtype: int64

In [66]:
# I WILL ADD 0 TO THE MISSING, AFTER I WILL ADD A VALUE THAT REPRESENT BETTER THIS FEATURES, LIKE MEAN FOR EXAMPLE
# I NOT WANT TO REMOVE THIS LINE BECAUSE EXISTS 5460 ITENS, IT IS A BIG AMOUNT OF INFORMATION
df.loc[df["garage"] == "", ["garage"]] = 0

## Value

In [67]:
# GET THE FIRST ELEMENT OF THE 
value_temp_list = df["value"].apply(lambda x : x.split(" ")[0])

In [68]:
# EXISTS SOME DIFFERENTS FORMATATION VALUES, THE R$, De AND Sob I WILL KEEP, THE OTHERS HAVE A SMALL NUMBER SO I WILL REMOVE THEM
value_temp_list.value_counts()

R$     30617
De        25
A         14
Sob        6
Name: value, dtype: int64

In [69]:
value_temp_list

0        R$
1        R$
2        R$
3        R$
4        R$
         ..
30662    R$
30663    R$
30664    R$
30665    R$
30666    R$
Name: value, Length: 30662, dtype: object

In [70]:
df.loc[value_temp_list == "value"].head(5)

,id,url,address,area,rooms,batrooms,garage,amenities,value,condominium,date


In [71]:
df.loc[value_temp_list == "A"].head(5)

,id,url,address,area,rooms,batrooms,garage,amenities,value,condominium,date
3257,2499477414,https://www.vivareal.com.br/imoveis-lancamento...,"Avenida Anita Garibaldi, 1282 - Cabral, Curiti...",78,3,2,2,NaN,A partir de\nR$ 615.000,NaN,2022-03-21 19:23:48
3827,2545075918,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Doutor Motta Júnior, 2600 - Centro, São Jo...",49,2,1,1,NaN,A partir de\nR$ 198.000,NaN,2022-03-21 19:28:41
4743,2549432009,https://www.vivareal.com.br/imoveis-lancamento...,"Rua João Américo de Oliveira, 922 - Cabral, Cu...",180,3,5,3,NaN,A partir de\nR$ 2.597.770,NaN,2022-03-21 19:36:13
4750,2520668057,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Moyses Marcondes, 53 - Juvevê, Curitiba - PR",105,3,3,2,NaN,A partir de\nR$ 1.030.000,NaN,2022-03-21 19:36:15
8854,2549435659,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Manoel Eufrásio, 1270 - Juvevê, Curitiba - PR",128,3,4,2,NaN,A partir de\nR$ 1.352.418,NaN,2022-03-21 20:08:38


In [72]:
print(df.loc[value_temp_list == "value"].shape[0])
print(df.loc[value_temp_list == "A"].shape[0])

0
14


In [73]:
# REMOVE ROWS WITH "value" AND "A"
df = df.loc[~(value_temp_list == "value")]
df = df.loc[~(value_temp_list == "A")]
print(df.loc[value_temp_list == "value"].shape[0])
print(df.loc[value_temp_list == "A"].shape[0])

0
0


In [74]:
df["value"] = cleanData(df, "value", " ", -1)

In [75]:
# REMOVE ALL LINES WITH THE VALUE Consulta
df = df.loc[~(df["value"] == "Consulta")]

In [76]:
# CHECK IF EXISTS SOME RENT APARTMENT
df.loc[df["value"].apply(lambda x : int(x.replace(".", "")) < 5000)]

,id,url,address,area,rooms,batrooms,garage,amenities,value,condominium,date


In [77]:
df["value"] = df["value"].apply(lambda x : int(x.replace(".", "")))

In [78]:
df["value"].value_counts().sum()

30642

## Condominium

In [79]:
# ALL NULL SET TO 0
df.loc[df["condominium"].isnull(), "condominium"] = "0"

In [80]:
df["condominium"] = cleanData(df, "condominium", " ", -1)

In [81]:
df["condominium"].value_counts()

0          15358
350          744
1            742
300          663
500          611
           ...  
1.153          1
1.114          1
1.552          1
1.693          1
400.000        1
Name: condominium, Length: 1433, dtype: int64

In [82]:
df["condominium"] = df["condominium"].apply(lambda x : x.replace(".", ""))

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30642 entries, 0 to 30666
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           30642 non-null  object
 1   url          30642 non-null  object
 2   address      30642 non-null  object
 3   area         30642 non-null  object
 4   rooms        30642 non-null  int64 
 5   batrooms     30642 non-null  object
 6   garage       30642 non-null  object
 7   amenities    24946 non-null  object
 8   value        30642 non-null  int64 
 9   condominium  30642 non-null  object
 10  date         30642 non-null  object
dtypes: int64(2), object(9)
memory usage: 2.8+ MB


## District

In [84]:
df["address"]

0        Rua Monsenhor Manoel Vicente, 597 - Água Verde...
1        Rua Professor Guido Straube, 775 - Vila Izabel...
2        Rua Monsenhor Manoel Vicente, 316 - Água Verde...
3        Rua Coronel Dulcídio, 1756 - Água Verde, Curit...
4         Avenida Iguaçu, 2960 - Água Verde, Curitiba - PR
                               ...                        
30662    Rua Francisco Derosso, 375 - Xaxim, Curitiba - PR
30663    Rua Antônio Rebelatto, 288 - Xaxim, Curitiba - PR
30664                                 Xaxim, Curitiba - PR
30665         Rua Abrão Winter, 580 - Xaxim, Curitiba - PR
30666          Rua David Tows, 1199 - Xaxim, Curitiba - PR
Name: address, Length: 30642, dtype: object

In [85]:
df["address"] = (df["address"].str.normalize('NFKD')
                             .str.encode('ascii', errors='ignore')
                             .str.decode('utf-8'))

In [86]:
bairros = df["address"].apply(lambda x : re.search(r"- ([A-Za-z]+ [A-Za-z]+)(,)", x))

In [87]:
result = []
for bairro in df["address"]:
    try:
        temp = re.search(r"- (.*),", bairro).group(1)
        result.append(temp.replace(" ","_"))
    except:
        temp = re.search(r"(.*),", bairro).group(1)
        result.append(temp.replace(" ", "_"))

In [88]:
df["bairros"] = result

In [89]:
df.loc[df["bairros"] == "1_-_Portao", "bairros"] = "Portao"

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30642 entries, 0 to 30666
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           30642 non-null  object
 1   url          30642 non-null  object
 2   address      30642 non-null  object
 3   area         30642 non-null  object
 4   rooms        30642 non-null  int64 
 5   batrooms     30642 non-null  object
 6   garage       30642 non-null  object
 7   amenities    24946 non-null  object
 8   value        30642 non-null  int64 
 9   condominium  30642 non-null  object
 10  date         30642 non-null  object
 11  bairros      30642 non-null  object
dtypes: int64(2), object(10)
memory usage: 3.0+ MB


In [91]:
df["area"] = df["area"].astype(int)
df["batrooms"] = df["batrooms"].astype(int)
df["garage"] = df["garage"].astype(int)
df["value"] = df["value"].astype(float)
df["condominium"] = df["condominium"].astype(float)

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30642 entries, 0 to 30666
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           30642 non-null  object 
 1   url          30642 non-null  object 
 2   address      30642 non-null  object 
 3   area         30642 non-null  int64  
 4   rooms        30642 non-null  int64  
 5   batrooms     30642 non-null  int64  
 6   garage       30642 non-null  int64  
 7   amenities    24946 non-null  object 
 8   value        30642 non-null  float64
 9   condominium  30642 non-null  float64
 10  date         30642 non-null  object 
 11  bairros      30642 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 3.0+ MB


## Amenities

In [93]:
df["amenities"] = (df["amenities"].str.normalize('NFKD')
                                  .str.encode('ascii', errors='ignore')
                                  .str.decode('utf-8')).str.replace(" ", "_").str.replace("_/", "").str.replace("_-", "")

In [94]:
df.loc[df["amenities"].isnull(), "amenities"] = "unknown"

In [95]:
df["bairros"].value_counts().index.sort_values()

Index(['Agua_Verde', 'Ahu', 'Alto_Boqueirao', 'Alto_da_Gloria',
       'Alto_da_Rua_XV', 'Atuba', 'Bacacheri', 'Bairro_Alto', 'Batel',
       'Bigorrilho', 'Boa_Vista', 'Bom_Retiro', 'Boqueirao', 'Cabral',
       'Cachoeira', 'Cajuru', 'Campina_do_Siqueira', 'Campo_Comprido',
       'Campo_de_Santana', 'Capao_Raso', 'Capao_da_Imbuia', 'Capela_Velha',
       'Centro', 'Centro_Civico', 'Champagnat', 'Cidade_Industrial',
       'Cristo_Rei', 'Ecoville', 'Fanny', 'Fazendinha', 'Guaira', 'Hauer',
       'Hugo_Lange', 'Iguacu', 'Jardim_Botanico', 'Jardim_Social', 'Juveve',
       'Lamenha_Grande', 'Lindoia', 'Merces', 'Mossungue', 'Novo_Mundo',
       'Ouro_Verde', 'Parolin', 'Pedro_Moro', 'Pilarzinho', 'Pinheirinho',
       'Portao', 'Prado_Velho', 'Reboucas', 'Santa_Candida',
       'Santa_Felicidade', 'Santa_Quiteria', 'Santo_Antonio', 'Santo_Inacio',
       'Sao_Francisco', 'Sao_Lourenco', 'Sao_Pedro', 'Seminario',
       'Sitio_Cercado', 'Tangua', 'Taruma', 'Tatuquara', 'Tingui', 'Ubera

In [96]:
df = pd.concat([df, df["amenities"].str.get_dummies(sep="\n")], axis=1)

In [97]:
# df = pd.concat([df, df["bairros"].str.get_dummies()], axis=1)

In [98]:
# df = df.drop(["...","amenities", "address", "id", "date", "bairros"], axis=1)
df = df.drop(["...","amenities", "address", "id", "date"], axis=1)

In [99]:
url = df["url"]
df_clean = df.drop("url", axis=1)

# Saving cleaning data

In [100]:
df.to_csv(path_or_buf="../../dataset/data_vivareal_clean.csv", index=False)